In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

In [0]:
df= spark.read.format("parquet")\
               .load("abfss://bronze@etedatabricksa.dfs.core.windows.net/Customers")


In [0]:
df.display()

In [0]:
df = df.drop("_rescued_data")

df.display()

In [0]:
df = df.withColumn("Domain",split(col("email"), "@").getItem(1))

df.display()

In [0]:
df_gmail = df.filter(col("Domain")=="gmail.com")
time.sleep(5)
df_gmail.display()

df_yahoo = df.filter(col("Domain")== "yahoo.com")
time.sleep(5)
df_yahoo.display()

df_hotmail =df.filter(col("Domain")=="hotmail.com")
time.sleep(5)
df_hotmail.display()

In [0]:
df = df.withColumn("full_name",concat(col('first_name'),lit(" "),col('last_name')))
df = df.drop('first_name','last_name')
df.display()

In [0]:
df.write.mode("append").format("delta").save("abfss://silver@etedatabricksa.dfs.core.windows.net/Customers")

In [0]:
%sql
CREATE SCHEMA DATABRICKS_CATALOG.silver

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_catalog.silver.customers_silver
USING DELTA
LOCATION 'abfss://silver@etedatabricksa.dfs.core.windows.net/Customers'

In [0]:
%sql
select * from databricks_catalog.silver.customers_silver
